# Data cleaning
In this notebook, we focus on cleaning the data set we extracted and extracting the most relevant attributes.

## Import the needed libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Import from local library
from helsinkihouse.config import data_path

%matplotlib inline
%load_ext autoreload
%autoreload 2  # reload modules before executing user code

## Read the data

In [3]:
housing = pd.read_excel(data_path, index_col=0)
housing.head()

,Link,Title,Address,Price,Size,Year
0,https://www.etuovi.com/kohde/21189890?haku=M16...,Kerrostalo | 2h + kt + s,"Rosendalinrinki 3, Tammisto, Vantaa",227000,49.5,2009
1,https://www.etuovi.com/kohde/7971959?haku=M160...,Rivitalo | 4h + kt + s,"Kanttiinitie 6 B4, Painiitty, Espoo",385900,92.0,2022
2,https://www.etuovi.com/kohde/21772126?haku=M16...,Kerrostalo | 3h + k,"Konalantie 4, Pitäjänmäki, Helsinki",205000,66.0,1963
3,https://www.etuovi.com/kohde/1287367?haku=M160...,"Rivitalo | 3-4h, k, kph / 2wc, khh, sauna, vh ...","Ripusuontie 64 A 2, Länsi-Pakila, Helsinki",545000,92.0,2020
4,https://www.etuovi.com/kohde/20553097?haku=M16...,Kerrostalo | 2h + k + kph + wc (yht.järj.muk. ...,"Kievarinraitti 2, Viherlaakso, Espoo",149000,56.0,1980


In [4]:
housing.shape

(5228, 6)

In [ ]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5228 entries, 0 to 5227
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Link         5228 non-null   object 
 1   Address      5228 non-null   object 
 2   Price        5228 non-null   int64  
 3   Size         5228 non-null   float64
 4   Year         5228 non-null   int64  
 5   House_type   5228 non-null   object 
 6   Description  5211 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 486.8+ KB


## Split information in different attributes
Note that the attributes 'Title' and 'Address' contain information that can still be subdivided in different attributes. This is what will be done in this section.



### Total number of rooms
First, let's look at 'Title'.

In [ ]:
housing.Title[3]

'Rivitalo | 3-4h, k, kph / 2wc, khh, sauna, vh 92 m2 + lämmin varasto 4 m2'

As we see, the first part of the attribute refers to the type of building (e.g. Rivitalo: Terraced house). The second part refers to the layout of the house (e.g. number of rooms) and other data. From the latter attribute, we will only retain the number of rooms as the other descriptions are not present in all data points and are in different formats which makes parsing them quite challenging.

In [ ]:
housing[["House_type", "Description"]] = housing.Title.str.split("|", expand=True)

In [ ]:
housing.head()

,Link,Title,Address,Price,Size,Year,House_type,Description
0,https://www.etuovi.com/kohde/21189890?haku=M16...,Kerrostalo | 2h + kt + s,"Rosendalinrinki 3, Tammisto, Vantaa",227000,49.5,2009,Kerrostalo,2h + kt + s
1,https://www.etuovi.com/kohde/7971959?haku=M160...,Rivitalo | 4h + kt + s,"Kanttiinitie 6 B4, Painiitty, Espoo",385900,92.0,2022,Rivitalo,4h + kt + s
2,https://www.etuovi.com/kohde/21772126?haku=M16...,Kerrostalo | 3h + k,"Konalantie 4, Pitäjänmäki, Helsinki",205000,66.0,1963,Kerrostalo,3h + k
3,https://www.etuovi.com/kohde/1287367?haku=M160...,"Rivitalo | 3-4h, k, kph / 2wc, khh, sauna, vh ...","Ripusuontie 64 A 2, Länsi-Pakila, Helsinki",545000,92.0,2020,Rivitalo,"3-4h, k, kph / 2wc, khh, sauna, vh 92 m2 + lä..."
4,https://www.etuovi.com/kohde/20553097?haku=M16...,Kerrostalo | 2h + k + kph + wc (yht.järj.muk. ...,"Kievarinraitti 2, Viherlaakso, Espoo",149000,56.0,1980,Kerrostalo,2h + k + kph + wc (yht.järj.muk. 2h + k)


In [ ]:
housing.drop("Title", axis=1, inplace=True)
housing.head()

,Link,Address,Price,Size,Year,House_type,Description
0,https://www.etuovi.com/kohde/21189890?haku=M16...,"Rosendalinrinki 3, Tammisto, Vantaa",227000,49.5,2009,Kerrostalo,2h + kt + s
1,https://www.etuovi.com/kohde/7971959?haku=M160...,"Kanttiinitie 6 B4, Painiitty, Espoo",385900,92.0,2022,Rivitalo,4h + kt + s
2,https://www.etuovi.com/kohde/21772126?haku=M16...,"Konalantie 4, Pitäjänmäki, Helsinki",205000,66.0,1963,Kerrostalo,3h + k
3,https://www.etuovi.com/kohde/1287367?haku=M160...,"Ripusuontie 64 A 2, Länsi-Pakila, Helsinki",545000,92.0,2020,Rivitalo,"3-4h, k, kph / 2wc, khh, sauna, vh 92 m2 + lä..."
4,https://www.etuovi.com/kohde/20553097?haku=M16...,"Kievarinraitti 2, Viherlaakso, Espoo",149000,56.0,1980,Kerrostalo,2h + k + kph + wc (yht.järj.muk. 2h + k)


In [ ]:
# Look what types of housing are present
housing.House_type.unique()

array(['Kerrostalo ', 'Rivitalo ', 'Paritalo ', 'Luhtitalo ',
       'Omakotitalo ', 'Paritalo', 'Erillistalo ', 'Omakotitalo',
       'Kerrostalo', 'Puutalo-osake ', 'Rivitalo'], dtype=object)

It seems some categories have an extra space at the end which is causing them to look like 2 different categories (e.g. 'Rivitalo ' and 'Rivitalo'). Let's fix that.

In [ ]:
housing.House_type = housing.House_type.str.replace(' ', '')
housing.House_type.unique()

array(['Kerrostalo', 'Rivitalo', 'Paritalo', 'Luhtitalo', 'Omakotitalo',
       'Erillistalo', 'Puutalo-osake'], dtype=object)

For the second part of "Title", we are interested in recovering the number of rooms (identified as #h). Let's try to do so now.

In [ ]:
housing["Total_rooms"] = housing.Description.str.split('h|H|-|k|mh|\(|,|x|\+', expand=True)[0]  # the '-' is for those where two numbers are specified (e.g. '3-4')
housing.head()

,Link,Address,Price,Size,Year,House_type,Description,Total_rooms
0,https://www.etuovi.com/kohde/21189890?haku=M16...,"Rosendalinrinki 3, Tammisto, Vantaa",227000,49.5,2009,Kerrostalo,2h + kt + s,2
1,https://www.etuovi.com/kohde/7971959?haku=M160...,"Kanttiinitie 6 B4, Painiitty, Espoo",385900,92.0,2022,Rivitalo,4h + kt + s,4
2,https://www.etuovi.com/kohde/21772126?haku=M16...,"Konalantie 4, Pitäjänmäki, Helsinki",205000,66.0,1963,Kerrostalo,3h + k,3
3,https://www.etuovi.com/kohde/1287367?haku=M160...,"Ripusuontie 64 A 2, Länsi-Pakila, Helsinki",545000,92.0,2020,Rivitalo,"3-4h, k, kph / 2wc, khh, sauna, vh 92 m2 + lä...",3
4,https://www.etuovi.com/kohde/20553097?haku=M16...,"Kievarinraitti 2, Viherlaakso, Espoo",149000,56.0,1980,Kerrostalo,2h + k + kph + wc (yht.järj.muk. 2h + k),2


In [ ]:
housing.Total_rooms.unique()

array([' 2', ' 4', ' 3', ' Y', ' 6', ' 1', ' 4 ', None, ' 5', ' 5 ',
       ' 3 ', ' auto', ' 7', ' PT 3', ' 6 ', ' Ka', ' K', ' 1 ', ' 2 ',
       ' ENNAKKOMARKKINOINNISSA', ' Ala', ' Valoisa ', ' Valoisa', ' ',
       ' KT 2', ' 8 ', ' Loft 1', ' O', ' 9', ' KT 3', ' Kolmio ', ' I',
       ' RT 3', ' Myymälä / lii', ' Valoisa LOFT tyylinen 1', ' 7 ',
       ' Ylimmän ', ' Parve', ' 2 ma', ' Erillistalo 5', ' Olo', ' Put',
       ' Kaivpuistossa ', ' Unii', ' o', ' Upeasti remontoitu ja put',
       ' Muuttovalmis rivitalo', ' Ylin ', ' 12 ', ' Pa',
       ' Avara ja valoisa ', ' 8', ' AK: o', ' Omalla tontilla ', ' PT',
       ' OKT 3', ' Kt', ' Talo järvinä', ' Neljännen ', ' Nyt ra', ' ylä',
       ' Avarapo', ' Vantaa As', ' Tupa', ' NYT ENNAKKOMARKKINOINNISSA',
       ' Valoisa saunallinen ', ' UUSI 4', ' Uusi 3', ' Esteetön',
       ' Enna', ' Uusia paritaloja Metsolaan', ' KT', ' autopai',
       ' Iso o', ' Klassinen 40', ' Talo 222 / 245 m2', ' Kompa', ' Uude',
       ' Kolmi

As we can see not all adds follow the same convention of writing first the number of rooms (although most do). We must deal with this cases where other information is provided and not the number of rooms. We also need to deal with the cases where the 

In [ ]:
housing.Total_rooms = housing.Total_rooms.str.replace(r'KT|RT|PT', '')  # remove characters for type of house

In [ ]:
housing.Total_rooms = housing.Total_rooms.str.replace(' ', '')  # remove empty spaces

In [ ]:
housing.Total_rooms = housing.Total_rooms.str.extract('(\d+)', expand=False)

In [ ]:
housing.Total_rooms.unique()

array(['2', '4', '3', nan, '6', '1', '5', '7', '8', '9', '12', '40',
       '222', '10', '112', '11'], dtype=object)

Although the problem is mostly fixed some specific values are clearly wrong (e.g. 222 rooms). Let's fix this on a one-to-one basis.

In [ ]:
housing[housing.Total_rooms=='222'].Link

3212    https://www.etuovi.com/kohde/20695302?haku=M16...
Name: Link, dtype: object

In [ ]:
housing.loc[housing.Total_rooms=='222', "Total_rooms"] = 6

In [ ]:
housing.Total_rooms.unique()

array(['2', '4', '3', nan, '6', '1', '5', '7', '8', '9', '12', '40', 6,
       '10', '112', '11'], dtype=object)

In [ ]:
housing[housing.Total_rooms=='40'].Link

3157    https://www.etuovi.com/kohde/21857207?haku=M16...
Name: Link, dtype: object

In [ ]:
housing.loc[housing.Total_rooms=='40', "Total_rooms"] = 4

In [ ]:
housing.Total_rooms.unique()

array(['2', '4', '3', nan, '6', '1', '5', '7', '8', '9', '12', 4, 6, '10',
       '112', '11'], dtype=object)

In [ ]:
housing[housing.Total_rooms=='112'].Link

4048    https://www.etuovi.com/kohde/1284881?haku=M160...
Name: Link, dtype: object

In [ ]:
housing.loc[housing.Total_rooms=='112', "Total_rooms"] = 5

In [ ]:
housing.Total_rooms.unique()

array(['2', '4', '3', nan, '6', '1', '5', '7', '8', '9', '12', 4, 6, '10',
       5, '11'], dtype=object)

### Longitude and Latitude

Now, we will focus on 'Address'. From this information, it would be useful to extract the longitude and latitude as it is very relevant for predicition and visualization.

In [ ]:
temp_df = pd.DataFrame({'name': housing["Address"]})
temp_df.tail()

,name
5223,"Ojalehdonkuja 2 A 41, Asola, Vantaa"
5224,"Laakavuorenkuja 1 A 5, Mellunmäki, Helsinki"
5225,"Pajuniitty 9a, Lippajärvi-Järvenperä, Espoo"
5226,"Koirasaarentie 50, Kruunuvuorenranta, Helsinki"
5227,"Koirasaarentie 50, Kruunuvuorenranta, Helsinki"


In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="adfgasdf")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
temp_df['location'] = temp_df['name'].apply(geocode)
temp_df['point'] = temp_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Karrintie 18 B 3, Puistola, Helsinki',), **{}).
Traceback (most recent call last):
  File "/usr/lib/python3.6/urllib/request.py", line 1325, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/usr/lib/python3.6/http/client.py", line 1281, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/lib/python3.6/http/client.py", line 1327, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.6/http/client.py", line 1276, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/lib/python3.6/http/client.py", line 1042, in _send_output
    self.send(msg)
  File "/usr/lib/python3.6/http/client.py", line 980, in send
    self.connect()
  File "/usr/lib/python3.6/http/client.py", line 1434, in connect
    super().connect()
  File "/usr/lib/python3.6/http/client.py", line 952, in connect
   

In [ ]:
temp_df.to_excel("temp.xls")

In [ ]:
temp_df.head()

,name,location,point
0,"Rosendalinrinki 3, Tammisto, Vantaa","(3, Rosendalinrinki, Tammisto, Aviapoliksen su...","(60.26879805, 24.968811530301256, 0.0)"
1,"Kanttiinitie 6 B4, Painiitty, Espoo","(Kanttiinitie, Painiitty, Lintuvaara, Suur-Lep...","(60.2374468, 24.8342875, 0.0)"
2,"Konalantie 4, Pitäjänmäki, Helsinki","(4 A, Konalantie, Reimarla, Pitäjänmäki, Länti...","(60.2257237, 24.85369199706898, 0.0)"
3,"Ripusuontie 64 A 2, Länsi-Pakila, Helsinki","(Ripusuontie, Länsi-Pakila, Pakila, Pohjoinen ...","(60.2472348, 24.9454876, 0.0)"
4,"Kievarinraitti 2, Viherlaakso, Espoo","(Kievarinraitti, Viherlaakso, Suur-Leppävaara,...","(60.2291669, 24.7422853, 0.0)"


Now, let's extract latitude, longitude and altitude from 'point'.

In [ ]:
temp_df[['Latitude', 'Longitude', 'Altitude']] = pd.DataFrame(temp_df.point.tolist())
temp_df.head()

,name,location,point,Latitude,Longitude,Altitude
0,"Rosendalinrinki 3, Tammisto, Vantaa","(3, Rosendalinrinki, Tammisto, Aviapoliksen su...","(60.26879805, 24.968811530301256, 0.0)",60.268798,24.968812,0.0
1,"Kanttiinitie 6 B4, Painiitty, Espoo","(Kanttiinitie, Painiitty, Lintuvaara, Suur-Lep...","(60.2374468, 24.8342875, 0.0)",60.237447,24.834287,0.0
2,"Konalantie 4, Pitäjänmäki, Helsinki","(4 A, Konalantie, Reimarla, Pitäjänmäki, Länti...","(60.2257237, 24.85369199706898, 0.0)",60.225724,24.853692,0.0
3,"Ripusuontie 64 A 2, Länsi-Pakila, Helsinki","(Ripusuontie, Länsi-Pakila, Pakila, Pohjoinen ...","(60.2472348, 24.9454876, 0.0)",60.247235,24.945488,0.0
4,"Kievarinraitti 2, Viherlaakso, Espoo","(Kievarinraitti, Viherlaakso, Suur-Leppävaara,...","(60.2291669, 24.7422853, 0.0)",60.229167,24.742285,0.0


Let's add the useful attributes (Latitude and Longitude) back to the cleaned data frame.

In [ ]:
housing.head()

,Link,Address,Price,Size,Year,House_type,Description,Total_rooms
0,https://www.etuovi.com/kohde/21189890?haku=M16...,"Rosendalinrinki 3, Tammisto, Vantaa",227000,49.5,2009,Kerrostalo,2h + kt + s,2
1,https://www.etuovi.com/kohde/7971959?haku=M160...,"Kanttiinitie 6 B4, Painiitty, Espoo",385900,92.0,2022,Rivitalo,4h + kt + s,4
2,https://www.etuovi.com/kohde/21772126?haku=M16...,"Konalantie 4, Pitäjänmäki, Helsinki",205000,66.0,1963,Kerrostalo,3h + k,3
3,https://www.etuovi.com/kohde/1287367?haku=M160...,"Ripusuontie 64 A 2, Länsi-Pakila, Helsinki",545000,92.0,2020,Rivitalo,"3-4h, k, kph / 2wc, khh, sauna, vh 92 m2 + lä...",3
4,https://www.etuovi.com/kohde/20553097?haku=M16...,"Kievarinraitti 2, Viherlaakso, Espoo",149000,56.0,1980,Kerrostalo,2h + k + kph + wc (yht.järj.muk. 2h + k),2


In [ ]:
housing = pd.concat([housing, temp_df[['Latitude', 'Longitude']]], axis=1)
housing.head()

,Link,Address,Price,Size,Year,House_type,Description,Total_rooms,Latitude,Longitude
0,https://www.etuovi.com/kohde/21189890?haku=M16...,"Rosendalinrinki 3, Tammisto, Vantaa",227000,49.5,2009,Kerrostalo,2h + kt + s,2,60.268798,24.968812
1,https://www.etuovi.com/kohde/7971959?haku=M160...,"Kanttiinitie 6 B4, Painiitty, Espoo",385900,92.0,2022,Rivitalo,4h + kt + s,4,60.237447,24.834287
2,https://www.etuovi.com/kohde/21772126?haku=M16...,"Konalantie 4, Pitäjänmäki, Helsinki",205000,66.0,1963,Kerrostalo,3h + k,3,60.225724,24.853692
3,https://www.etuovi.com/kohde/1287367?haku=M160...,"Ripusuontie 64 A 2, Länsi-Pakila, Helsinki",545000,92.0,2020,Rivitalo,"3-4h, k, kph / 2wc, khh, sauna, vh 92 m2 + lä...",3,60.247235,24.945488
4,https://www.etuovi.com/kohde/20553097?haku=M16...,"Kievarinraitti 2, Viherlaakso, Espoo",149000,56.0,1980,Kerrostalo,2h + k + kph + wc (yht.järj.muk. 2h + k),2,60.229167,24.742285


In [ ]:
housing.shape

(5228, 10)

## Export cleaned data set

In [ ]:
housing.to_excel("helsinki_house_price_cleaned.xls")